In [1]:
import nltk
import math
import random
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.util import ngrams
from math import log, exp
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split
tokenizer = nltk.data.load("tokenizers/punkt/english.pickle")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/shreyas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
with open('shrelock.txt', 'r') as fin:
    text = fin.read()
print(text)

Project Gutenberg’s Alice’s Adventures in Wonderland, by Lewis Carroll

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org


Title: Alice’s Adventures in Wonderland

Author: Lewis Carroll

Posting Date: June 25, 2008 [EBook #11]
Release Date: March, 1994
Last Updated: October 6, 2016

Language: English

Character set encoding: UTF-8

*** START OF THIS PROJECT GUTENBERG EBOOK ALICE’S ADVENTURES IN WONDERLAND ***










ALICE’S ADVENTURES IN WONDERLAND

Lewis Carroll

THE MILLENNIUM FULCRUM EDITION 3.0




CHAPTER I. Down the Rabbit-Hole

Alice was beginning to get very tired of sitting by her sister on the
bank, and of having nothing to do: once or twice she had peeped into the
book her sister was reading, but it had no pictures or conversations in
it, ‘and what is the use of a book,’ thou

In [3]:
token_sent = [ word_tokenize(line) for line in tokenizer.tokenize(text)]
token_list = []
for line in token_sent:
    sent = [ word for word in line if word.isalpha()]
    token_list.append([ word.lower() for word in sent ])

token_list = [ ["<s>"] + line + ["</s>"] for line in token_list ]
print(token_list)
token_sent_train ,token_sent_test = train_test_split(token_list,test_size=0.8)
#print(token_sent_train)

[['<s>', 'project', 'gutenberg', 's', 'alice', 's', 'adventures', 'in', 'wonderland', 'by', 'lewis', 'carroll', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '</s>'], ['<s>', 'you', 'may', 'copy', 'it', 'give', 'it', 'away', 'or', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'title', 'alice', 's', 'adventures', 'in', 'wonderland', 'author', 'lewis', 'carroll', 'posting', 'date', 'june', 'ebook', 'release', 'date', 'march', 'last', 'updated', 'october', 'language', 'english', 'character', 'set', 'encoding', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'alice', 's', 'adventures', 'in', 'wonderland', 'alice', 's', 'adventures', 'in', 'wonderland', 'lewis', 'carroll', 'the', 'millennium', 'fulcrum', 'edition', 'chapter', 'i', '</s>'], ['<s>', 'down', 'the', 'alice', 'was', 'beginnin

In [4]:
# Check Counts

ngram_list = {1:{},2:{},3:{},4:{}}
for line in token_list:
    for j in range(len(line)):
        ngram_list[1][line[j]] = ngram_list[1].get(line[j], 0) + 1
    for j in range(len(line)-1):
        bi = line[j] + " " + line[j+1]
        ngram_list[2][bi] = ngram_list[2].get(bi, 0) + 1
    for j in range(len(line)-2):
        tri = line[j] + " " + line[j+1] + " " + line[j+2]
        ngram_list[3][tri] = ngram_list[3].get(tri, 0) + 1
    for j in range(len(line)-3):
        quad = line[j] + " " + line[j+1] + " " + line[j+2] + " " + line[j+3]
        ngram_list[4][quad] = ngram_list[4].get(quad, 0) + 1

print(ngram_list[1])

{'<s>': 1093, 'project': 87, 'gutenberg': 30, 's': 213, 'alice': 403, 'adventures': 11, 'in': 428, 'wonderland': 8, 'by': 77, 'lewis': 4, 'carroll': 4, 'this': 181, 'ebook': 9, 'is': 131, 'for': 178, 'the': 1813, 'use': 29, 'of': 628, 'anyone': 5, 'anywhere': 3, 'at': 227, 'no': 100, 'cost': 4, 'and': 934, 'with': 227, 'almost': 8, 'restrictions': 2, 'whatsoever': 2, '</s>': 1093, 'you': 479, 'may': 28, 'copy': 12, 'it': 601, 'give': 16, 'away': 28, 'or': 155, 'under': 22, 'terms': 22, 'license': 15, 'included': 3, 'online': 4, 'title': 1, 'author': 1, 'posting': 1, 'date': 4, 'june': 1, 'release': 1, 'march': 34, 'last': 34, 'updated': 2, 'october': 1, 'language': 1, 'english': 7, 'character': 2, 'set': 23, 'encoding': 1, 'start': 3, 'millennium': 1, 'fulcrum': 1, 'edition': 2, 'chapter': 12, 'i': 543, 'down': 102, 'was': 357, 'beginning': 15, 'to': 804, 'get': 46, 'very': 145, 'tired': 7, 'sitting': 10, 'her': 248, 'sister': 9, 'on': 202, 'bank': 3, 'having': 10, 'nothing': 33, 'do':

In [5]:
def nCr(n,r):
    f = n
    for i in range(1,r):
        f *= (n-i)/(i+1)
    return f

type_num = len(ngram_list[1])
print("        {0: <10} {1: <20} {2:<10}".format("Actual", "Possible", "% left"))
print("        {0: <10} {1: <20} {2:<10}".format("------", "--------", "------"))

print(1, "gram  {0: <10} {1: <20} {2:<10}".format(type_num, type_num, 0.0))

for i in range(2,5):
    temp = type_num ** i
    temp2 = len(ngram_list[i])
    print(i, "gram  {0: <10} {1: <20} {2:<10}".format(temp2, temp, (temp-temp2)*100/temp))

        Actual     Possible             % left    
        ------     --------             ------    
1 gram  2920       2920                 0.0       
2 gram  15788      8526400              99.81483392756614
3 gram  24780      24897088000          99.99990047028793
4 gram  27077      72699496960000       99.9999999627549


In [6]:
mle_list = {1:{}, 2:{}, 3:{}, 4:{}}

# unigrams
total_uni = 0
for token in ngram_list[1]:
    total_uni += ngram_list[1][token]
for uni in ngram_list[1]:
    mle_list[1][uni] = ngram_list[1][uni]/total_uni

# bigrams
for bi in ngram_list[2]:
    mle_list[2][bi] = ngram_list[2][bi]/ngram_list[1][bi.split()[0]]

# trigrams
for tri in ngram_list[3]:
    mle_list[3][tri] = ngram_list[3][tri]/ngram_list[2][' '.join(tri.split()[:2])]

# quadgrams
for quad in ngram_list[4]:
    mle_list[4][quad] = ngram_list[4][quad]/ngram_list[3][' '.join(quad.split()[:3])]

In [7]:
def probability_sentence(sentence, model_name, ngram_list):
    n_list = []
    sent = sentence.split()
    for j in range (len(sent)-model_name+1):
        n_temp = sent[j].lower()
        for i in range(model_name-1):
            n_temp += " " + sent[j+i+1].lower()
        n_list.append([n_temp, mle_list[model_name][n_temp]])
    
    log_prob = 0
    for n_prob in n_list:
        log_prob += math.log(n_prob[1])
    
    return math.exp(log_prob), n_list

probability_sentence("Either the well was very deep", 2, ngram_list)

(1.937328592128443e-09,
 [['either the', 0.18181818181818182],
  ['the well', 0.001654715940430226],
  ['well was', 0.016666666666666666],
  ['was very', 0.028011204481792718],
  ['very deep', 0.013793103448275862]])

In [8]:
def generate_sentence(n_gram, mle_list):
    n_list = []
    n_prob = []

    for n_token, mle_prob in mle_list[n_gram].items():
        if '<s>' in n_token:
            n_list.append(n_token)
            n_prob.append(mle_prob)

    start = random.choices(n_list,n_prob)[0]
    end = start.split()[1:]

    i = 0
    
    while '</s>' not in end:
        n_list = []
        n_prob = []
        for n_token, mle_prob in mle_list[n_gram].items():
            if end == n_token.split()[:-1]:
                n_list.append(n_token)
                n_prob.append(mle_prob)
        
        if i > 25 or n_list == []:
            start = generate_sentence(n_gram, mle_list)
            break
        
        choice = random.choices(n_list,n_prob)[0]
        #print(choice)
        start += " " + choice.split()[-1]
        end = choice.split()[1:]
        
        i += 1
        
    return start

print(generate_sentence(4, mle_list))

<s> all right so far said the king looking round the court and got behind him and very soon finished it off </s>


In [9]:
# Work only on training set
ngram_list_test = {1:{},2:{}}

for line in token_sent_train:
    for j in range(len(line)):
        q[1][line[j]] = ngram_list_test[1].get(line[j], 0) + 1
    for j in range(len(line)-1):
        bi = line[j] + " " + line[j+1]
        ngram_list_test[2][bi] = ngram_list_test[2].get(bi, 0) + 1

mle_list_test = {1:{}, 2:{}}

# unigrams
total_uni = 0
for token in ngram_list_test[1]:
    total_uni += ngram_list_test[1][token]
for uni in ngram_list_test[1]:
    mle_list_test[1][uni] = ngram_list_test[1][uni]/total_uni

# bigrams
for bi in ngram_list_test[2]:
    mle_list_test[2][bi] = ngram_list_test[2][bi]/ngram_list_test[1][bi.split()[0]]
    
#print(ngram_list_test[2])

In [10]:
unigrams = list(ngram_list[1].keys())
type_num = len(ngram_list[1])

add_one = {}
for i in range(len(unigrams)):
    for j in range(len(unigrams)):
        if i == "</s>":
            break
        add_one[unigrams[i] + " " + unigrams[j]] = (ngram_list_test[2].get(unigrams[i] + " " + unigrams[j], 0) + 1)*(ngram_list_test[1].get(unigrams[i], 1))/(ngram_list_test[1].get(unigrams[i], 1) + type_num)

In [11]:
add_one_mle = {}
for bi in add_one:
    add_one_mle[bi] = add_one[bi]/(ngram_list_test[1].get(unigrams[i], 1))

In [12]:
count = 0

for bi in ngram_list_test[2]:
    if ngram_list_test[2][bi] - add_one[bi] > 20:
        print(bi, ngram_list_test[2][bi], add_one[bi])
        count += 1
    if count == 4:
        break

in the 27 0.9629629629629629
of the 39 1.7550753110674526
said the 43 1.3722627737226278
<s> i 23 1.6673040152963672


In [13]:
bi_count = {}
good_turing_count = []
total_count = 0

for count in ngram_list_test[2].values():
    bi_count[count] = bi_count.get(count, 0) + count
    total_count += count
bi_count[0] = type_num**2 - total_count

#print(bi_count)
diff = []
for i in range(10):
    good_turing_count.append(((i+1)*bi_count[i+1])/bi_count[i])
    diff.append(i - good_turing_count[i]) 
    print(i, '\t', diff[i])

print('\n', sum(diff[:7])/len(diff[:7]))

0 	 -0.0004532921309331151
1 	 0.506991196271362
2 	 0.5346638655462186
3 	 0.2129032258064516
4 	 1.6080246913580245
5 	 2.9096774193548387
6 	 -1.2592592592592595
7 	 -2.1428571428571423
8 	 1.671875
9 	 -2.1111111111111107

 0.6446496924209575


In [14]:
def curve(n,k,b):
     return k*(n**b)
    
popt, pcov = curve_fit(curve, list(range(2,10)), good_turing_count[2:10])
k = popt[0]
b = popt[1]

In [15]:
good_turing = {}
unigrams = list(ngram_list[1].keys())
type_num = len(ngram_list[1])

for i in range(len(unigrams)):
    for j in range(len(unigrams)):
        if unigrams[i] == "</s>":
            break
        bi = unigrams[i] + " " + unigrams[j]
        if bi not in ngram_list_test[2]:
            good_turing[bi] = good_turing_count[0]
        elif 0 < ngram_list_test[2][bi] < 10:
            good_turing[bi] = good_turing_count[ngram_list_test[2][bi]]
        else:
            good_turing[bi] = k*(ngram_list_test[2][bi]**b)

In [16]:
good_turing_mle = {}
for bi in good_turing:
    good_turing_mle[bi] = good_turing[bi]/ngram_list_test[2].get(bi.split()[0], 1)

In [18]:
# Perplexity for Add One

num_bi = 0
per_one = 1

for line in token_sent_train:
    num_bi += len(line) - 1

for line in token_sent_train:
    for j in range(len(line) - 1):
        k = (1/add_one_mle[line[j] + " " + line[j+1]])    
        per_one *=  math.pow(k,1.0/num_bi)
print(per_one)

51.24918497780626


In [19]:
# Perplexity for Good Turing

num_bi = 0
per_good = 1

for line in token_sent_train:
    num_bi += len(line) - 1

for line in token_sent_train:
    for j in range(len(line) - 1):
        k = (1/good_turing_mle[line[j] + " " + line[j+1]])    
        print(k)
        per_good *=  math.pow(k,1.0/num_bi)
print(per_good)

0.04424083042927289
0.6824372759856632
2.028361344537815
0.07073060646475882
2.028361344537815
2.028361344537815
2.028361344537815
0.3587962962962963
2.028361344537815
2.028361344537815
2.028361344537815
0.4180645161290322
0.6824372759856632
0.4180645161290322
2.028361344537815
2.028361344537815
2.028361344537815
0.6824372759856632
0.4180645161290322
0.3587962962962963
0.3587962962962963
0.6824372759856632
0.3587962962962963
2.028361344537815
0.6824372759856632
0.01818126414949465
0.6824372759856632
2.028361344537815
0.6824372759856632
2.028361344537815
0.07073060646475882
0.07073060646475882
2.028361344537815
2.028361344537815
0.109375
2.028361344537815
2.028361344537815
0.04424083042927289
2.028361344537815
0.6824372759856632
0.1377551020408163
2.028361344537815
2.028361344537815
2.028361344537815
0.028628903725662465
0.6824372759856632
2.028361344537815
2.028361344537815
0.4180645161290322
0.6824372759856632
0.3587962962962963
2.028361344537815
2.028361344537815
2.028361344537815
2.

2.028361344537815
0.6824372759856632
2.028361344537815
2.028361344537815
2.028361344537815
2.028361344537815
0.6824372759856632
0.4180645161290322
0.4180645161290322
2.028361344537815
0.6824372759856632
2.028361344537815
2.028361344537815
2.028361344537815
2.028361344537815
2.028361344537815
0.00899211662494675
0.6824372759856632
0.6824372759856632
2.028361344537815
0.4180645161290322
2.028361344537815
0.6824372759856632
0.062006533111539475
0.6824372759856632
2.028361344537815
0.6824372759856632
0.4180645161290322
2.028361344537815
2.028361344537815
0.3587962962962963
2.028361344537815
2.028361344537815
0.4180645161290322
0.4180645161290322
0.4180645161290322
2.028361344537815
0.4180645161290322
0.07073060646475882
2.028361344537815
2.028361344537815
2.028361344537815
0.3587962962962963
0.3587962962962963
0.3587962962962963
2.028361344537815
2.028361344537815
0.6824372759856632
0.3587962962962963
2.028361344537815
2.028361344537815
2.028361344537815
0.023172642881875045
0.418064516129